# Week 3 Assignment: Coursera Capstone

In [44]:
from bs4 import BeautifulSoup
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pandas as pd
import numpy as np

# Part 1: Building Data Frame

In [45]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(url)

In [46]:
c = result.content
soup = BeautifulSoup(c,"html.parser")

In [47]:
My_table = soup.find('table',{'class':'wikitable sortable'})


In [48]:
entries =My_table.findAll("td")
x=0
list0 =[]
list1 = []
list2 =[]


for i in entries:
    #
    x+=1
    y = x%3
    if y == 0:
        list0.append(i.text)
    if y ==1:
        list1.append(i.text)
    if y ==2:
        list2.append(i.text)
        

In [49]:
neighborhood_list =[] #starts empty list to hold string values
for string in list0:
    neighborhood_list.append(string.replace('\n','')) #removed end \n from each string
neighborhood_list[0:5]

['Not assigned',
 'Not assigned',
 'Parkwoods',
 'Victoria Village',
 'Harbourfront']

In [50]:
postal_list = list1.copy()
borough_list = list2.copy()

In [51]:
dfdata = {'Postal Code':postal_list, 'Boroughs':borough_list, 'Neighborhood':neighborhood_list}
df = pd.DataFrame( data =dfdata)
df.head()

,Postal Code,Boroughs,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [52]:
df.drop(df[df.Boroughs == 'Not assigned'].index,inplace =True) #drop boroughs with not assigned value
df

,Postal Code,Boroughs,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [53]:

df.loc[df['Neighborhood']=="Not assigned",'Neighborhood' ]= df.loc[df['Neighborhood']=="Not assigned",'Boroughs']
df.head(10) #look at index 8 to verify

,Postal Code,Boroughs,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [54]:
import json 
import folium

In [55]:
df1=df.groupby("Postal Code").agg(lambda x:', '.join(set(x)))

In [56]:
df1.shape

(103, 2)

# Part 2: Getting Location Data

In [57]:
geo=pd.read_csv("https://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
df1['Latitude']=geo['Latitude'].values #adding latittude for each postal code
df1['Longitude']=geo['Longitude'].values

df1.head()

,Boroughs,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [59]:
center_lat=df1.Latitude.mean()
center_long=df1.Longitude.mean()

# create map of toronto centered on the average locations, using latitude and longitude values
map_toronto = folium.Map(location=[center_lat, center_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Boroughs'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,).add_to(map_toronto)  
    
map_toronto

# Part 3: Analysis

In [60]:
import matplotlib.cm as cm
import matplotlib.colors as colors


In [61]:


CLIENT_ID = 'VYCOUITIGXOONWV3KJ34HKMDN2IAXOLBLRKF0CRIJSXXR1SJ' # your Foursquare ID
CLIENT_SECRET = 'H0ATIVHEOUNIBDCHAVN1YOT2K5GLUT4QT1HNO3VLPU00EIKB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VYCOUITIGXOONWV3KJ34HKMDN2IAXOLBLRKF0CRIJSXXR1SJ
CLIENT_SECRET:H0ATIVHEOUNIBDCHAVN1YOT2K5GLUT4QT1HNO3VLPU00EIKB


In [62]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    center_lat, 
    center_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VYCOUITIGXOONWV3KJ34HKMDN2IAXOLBLRKF0CRIJSXXR1SJ&client_secret=H0ATIVHEOUNIBDCHAVN1YOT2K5GLUT4QT1HNO3VLPU00EIKB&v=20180605&ll=43.70460773398059,-79.39715291165048&radius=500&limit=100'

In [63]:
results = requests.get(url).json()
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [64]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [65]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Zezafoun Syrian Cousine,Restaurant,43.702667,-79.397120
1,The Keg Steakhouse & Bar,Steakhouse,43.705464,-79.398085
2,Starbucks,Coffee Shop,43.706940,-79.396908
3,Yonge Eglinton Square,Plaza,43.706864,-79.398751
4,Loblaws,Supermarket,43.707412,-79.394909


In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude'],
                                 radius = 250
                                  )


Rouge, Malvern
Port Union, Rouge Hill, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Kennedy Park, Ionview
Golden Mile, Oakridge, Clairlea
Scarborough Village West, Cliffcrest, Cliffside
Cliffside West, Birch Cliff
Dorset Park, Scarborough Town Centre, Wexford Heights
Wexford, Maryvale
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Steeles East, Agincourt North, Milliken, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm, Oriole, Fairview
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Downsview North, Bathurst Manor, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
River

In [68]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,...,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

In [70]:
toronto_grouped.shape

(77, 182)

In [71]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0                   Sandwich Place   1.0
1                      Yoga Studio   0.0
2                        Nightclub   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Moroccan Restaurant   0.0


----Alderwood, Long Branch----
          venue  freq
0  Dance Studio  0.25
1   Pizza Place  0.25
2      Pharmacy  0.25
3   Coffee Shop  0.25
4   Yoga Studio  0.00


----Bathurst Quay, South Niagara, Railway Lands, Harbourfront West, Island airport, King and Spadina, CN Tower----
                             venue  freq
0            Performing Arts Venue   1.0
1                      Yoga Studio   0.0
2                        Nightclub   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Moroccan Restaurant   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.14
1    Sushi Restaurant  0.14
2  Italian Restaurant  0.14
3     Thai Restaurant  0.07
4        Liquor Store  0

                             venue  freq
0                       Playground  0.33
1                            Field  0.33
2                          Dog Run  0.33
3                        Nightclub  0.00
4  Molecular Gastronomy Restaurant  0.00


----Lawrence Heights, Lawrence Manor----
                    venue  freq
0          Clothing Store  0.43
1           Women's Store  0.14
2                Boutique  0.14
3  Furniture / Home Store  0.14
4             Event Space  0.14


----Lawrence Park----
                             venue  freq
0                             Lake   1.0
1                      Yoga Studio   0.0
2  Molecular Gastronomy Restaurant   0.0
3              Moroccan Restaurant   0.0
4                            Motel   0.0


----Leaside----
                 venue  freq
0          Coffee Shop  0.11
1           Restaurant  0.11
2  Sporting Goods Shop  0.11
3       Breakfast Spot  0.11
4       Clothing Store  0.11


----Little Portugal, Trinity----
              venue  f



----Victoria Village----
                   venue  freq
0            Pizza Place  0.25
1  Portuguese Restaurant  0.25
2           Intersection  0.25
3            Coffee Shop  0.25
4           Noodle House  0.00


----Westmount----
                             venue  freq
0                      Pizza Place   0.5
1                      Coffee Shop   0.5
2                      Yoga Studio   0.0
3                        Nightclub   0.0
4  Molecular Gastronomy Restaurant   0.0


----Weston----
                             venue  freq
0                             Park   1.0
1                      Yoga Studio   0.0
2                        Nightclub   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Moroccan Restaurant   0.0


----Woburn----
                             venue  freq
0                Korean Restaurant   1.0
1                     Noodle House   0.0
2  Molecular Gastronomy Restaurant   0.0
3              Moroccan Restaurant   0.0
4                            Motel  

In [72]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Sandwich Place,Women's Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,"Alderwood, Long Branch",Pharmacy,Dance Studio,Coffee Shop,Pizza Place,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
2,"Bathurst Quay, South Niagara, Railway Lands, H...",Performing Arts Venue,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Pub,Liquor Store,Sandwich Place,Restaurant,Juice Bar,Thai Restaurant
4,Berczy Park,Steakhouse,Liquor Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


# Clustering 
## Set 3 clusters in order to have emaningful groups

In [74]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 0, 1])

In [75]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_data =df1
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Boroughs,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Korean Restaurant,Women's Store,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Lounge,Thai Restaurant,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


# Cluster Map

In [76]:
# create map
map_clusters = folium.Map(location=[center_lat, center_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if np.isnan(cluster) :
            cluster = 0
            exit
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1: Retail and Restaurant Neighborhoods

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M1N,"Cliffside West, Birch Cliff",Pizza Place,Café,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M4A,Victoria Village,Pizza Place,Portuguese Restaurant,Intersection,Coffee Shop,Dumpling Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M5G,Central Bay Street,Coffee Shop,Café,Spa,Pharmacy,Smoothie Shop,Pizza Place,Deli / Bodega,Bookstore,Italian Restaurant,Sandwich Place
M5R,"The Annex, Yorkville, North Midtown",Coffee Shop,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M7R,Canada Post Gateway Processing Centre,Coffee Shop,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M8W,"Alderwood, Long Branch",Pharmacy,Dance Studio,Coffee Shop,Pizza Place,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
M9L,Humber Summit,Pizza Place,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M9P,Westmount,Coffee Shop,Pizza Place,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


## Cluster 2: Park Neighborhoods

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M1G,Woburn,Korean Restaurant,Women's Store,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M1H,Cedarbrae,Hakka Restaurant,Lounge,Thai Restaurant,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M1J,Scarborough Village,Playground,Food,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
M1L,"Golden Mile, Oakridge, Clairlea",Business Service,Women's Store,Food,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M1M,"Scarborough Village West, Cliffcrest, Cliffside",American Restaurant,Intersection,Motel,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
M1S,Agincourt,Sandwich Place,Women's Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
M1T,"Clarks Corners, Sullivan, Tam O'Shanter",Shopping Mall,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M1W,"Steeles West, L'Amoreaux West",Chinese Restaurant,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Cosmetics Shop,Coffee Shop,Sandwich Place,Thrift / Vintage Store,Pharmacy,Grocery Store
M2H,Hillcrest Village,Pool,Golf Course,Mediterranean Restaurant,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


## Cluster 3: Playground Neighborhoods

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,
M2P,York Mills West,Bank,Park,Women's Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M3A,Parkwoods,Park,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M4J,East Toronto,Park,Bus Stop,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M4W,Rosedale,Building,Park,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
M9N,Weston,Park,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
M9R,"Martin Grove Gardens, St. Phillips, Kingsview ...",Park,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Shape of dataframe

In [80]:
df1.shape

(103, 4)